Linea Base
----------

Usando las letras en expañol entrenar un sistema que devuelva una representacion en una dimension baja, despues entrenar un clasificador con esas representaciones.

Hay distintas opciones para obtener representaciones de las letras, una puede ser entrenando nuevos word vectors y despues sumarlos para obtener la representacion de la letra. La primera que vamos a usar es countvectorizer y luego PCA o SVD para reducir dimension (tambien podría ser T-sne).

Para entrenar word vectors con gensim: https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
Otro, con representacion de baja dimension: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
Jurafsky capitulo 16 (representacion usando SVD): https://web.stanford.edu/~jurafsky/slp3/16.pdf

In [1]:
import os
import json
import pickle
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

stop_words = ["de","la", "que","el","en","y","a","los","del","se"]

Se intentará usar LDA para transformar las letras (800k) en una representación vectorial para luego entrenar un clasificador y comparar el desempeño. Se realiza incialmente para una dimension de 10.

In [3]:
dimentions = 10
folder = 'data/test-full-lyrics/es'

vectorizer = HashingVectorizer(stop_words=stop_words, non_negative=True)#, alternate_sign=False)
dim_reduce = LatentDirichletAllocation(dimentions, random_state=42)
count = 0
spanish_lyrics = []
for filename in os.listdir(folder):
    f = open(folder+'/'+filename)
    lyric = json.load(f)
    spanish_lyrics.append("\n".join(lyric)) # falta un " " ?

    # We train LDA in batches
    if count == 5000:
        X_train = vectorizer.transform(spanish_lyrics)
        dim_reduce.partial_fit(X_train)
        count = 0
    count += 1
pickle.dump(dim_reduce, open('data/lda.p', 'wb'))

In [39]:
from todotango_cleaner import clean_dates

#vectorizer = HashingVectorizer(stop_words=stop_words, non_negative=True)
vectorizer = HashingVectorizer(stop_words=stop_words, non_negative=True, n_features=4000)
#dim_reduce = pickle.load(open('data/lda.p'))
years = clean_dates.get_years()


tango_lyrics_file = 'data/clean/lyrics.json'
tango_lyrics = json.load(open(tango_lyrics_file))

X = []
y = []
y_orig = []
for work_id in tango_lyrics.keys():  
    if int(work_id) in years:
        if years[int(work_id)] < 2017:
            y_orig.append(years[int(work_id)])
            y.append(years[int(work_id)] > 1950)
            X.append(tango_lyrics[work_id])
    
X_vect = vectorizer.transform(X)
#X_transf = dim_reduce.transform(X_vect)


Ahora se compara la clasificacion usando los vectores generados a partir de la transformacion contra los vectores originales del HashingVectorizer

In [11]:
clf = SVC(kernel='linear')
scores = cross_val_score(clf, X_vect, y, cv=5)
print scores

[ 0.76716918  0.72673931  0.73260687  0.72590109  0.73154362]


In [74]:
clf = SVC(kernel='linear')
scores = cross_val_score(clf, X_transf, y, cv=5)
#y_pred = cross_val_predict(clf, X_transf, y, cv=5)
#conf_mat = confusion_matrix(y, y_pred)
print scores

[ 0.51758794  0.51802179  0.51802179  0.51802179  0.51761745]


In [75]:
from collections import Counter
cnt = Counter(y)

In [76]:
for k in cnt.keys():
    print k, float(cnt[k])/len(y)

False 0.517854149204
True 0.482145850796


*** Regresión ***
=================

In [40]:
#from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
#from sklearn.linear_model import SGDRegressor

clf = SVR('linear')
scores = cross_val_score(clf, X_vect, y_orig, cv=3, scoring='neg_mean_absolute_error')

print scores

[-19.87076785 -19.73238833 -18.93854635]
